In [32]:

from pprint import pprint
from re import T
import torchvision.transforms as transforms  #图像预处理工具
import torch
import numpy as np
from PIL import Image
from matplotlib.pyplot import axes
from sympy import true
from torchvision.datasets import ImageFolder
from click.core import batch
from matplotlib import pyplot as plt
import os

from torch.distributions import transform_to
from torch.utils.data import DataLoader, Dataset
from torch.xpu import device


In [33]:
#自定义数据类型
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.main_dir = image_dir
        self.transform = transform
        self.image_names=os.listdir(image_dir)  #获取目录下所有文件名
        self.image_filenames = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_names)

#传入图片id，获取数据元素
    def __getitem__(self, idx):
        #根据索引号，构建图片的完整路径
        img_loc= os.path.join(self.main_dir, self.image_filenames[idx])
        image = Image.open(img_loc).convert('RGB')

        #利用transform转换成tensor
        if self.transform:
            image = self.transform(image)
        #抛出异常
        else:
            raise ValueError("Transform is not defined")

        #向原始图像中增加随机噪声
        noisy_factor = 0.5
        noisy_image = image + noisy_factor * torch.randn(*image.shape)
        noisy_image = torch.clamp(noisy_image, 0., 1.)
#返回噪声图片和原始图片
        return noisy_image, image




In [34]:
from torchvision.transforms import Compose
import torchvision.transforms as transforms  #图像预处理工具
 #测试主流程
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((68,68)),
    transforms.ToTensor()
])
#写代码时没有注意到中括号

#创建数据集
full_dataset=ImageDataset(image_dir ='../common/dataset/', transform=transform)

print(len(full_dataset))


24853


In [35]:
import torch
from torch.utils.data import random_split
#划分数据集
#train_dataset, test_dataset = random_split(full_dataset[0.75,0.25])
train_dataset, test_dataset = random_split(full_dataset, [0.75, 0.25])
print(len(train_dataset))
print(len(test_dataset))

18640
6213


In [36]:
#创建dataLoder
batch_size=64
train_loder=DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True

)

In [37]:
test_loder=DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,

)

In [38]:
for x,y in train_loder:
   print(f"x: {x}, x shape: {x.shape}, x device: {x.device}")
   print(f"y: {y}, y shape: {y.shape}, y device: {y.device}")
   break

x: tensor([[[[1.0000, 1.0000, 0.4207,  ..., 0.6925, 0.4715, 0.1095],
          [0.8190, 1.0000, 0.5193,  ..., 0.5236, 0.5242, 1.0000],
          [0.2606, 0.6831, 0.2847,  ..., 0.2032, 1.0000, 1.0000],
          ...,
          [0.8317, 1.0000, 0.4683,  ..., 1.0000, 0.2668, 1.0000],
          [1.0000, 0.7080, 1.0000,  ..., 1.0000, 1.0000, 0.9963],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[1.0000, 0.2627, 0.8472,  ..., 1.0000, 1.0000, 0.6547],
          [1.0000, 1.0000, 1.0000,  ..., 0.2744, 0.4890, 0.8258],
          [0.7420, 1.0000, 0.6970,  ..., 1.0000, 0.9862, 1.0000],
          ...,
          [1.0000, 1.0000, 0.7487,  ..., 1.0000, 0.8059, 0.9746],
          [1.0000, 0.0436, 1.0000,  ..., 0.0000, 0.2334, 0.5922],
          [1.0000, 0.0286, 0.9560,  ..., 0.9411, 1.0000, 0.5546]],

         [[0.7559, 1.0000, 0.9247,  ..., 1.0000, 1.0000, 1.0000],
          [0.4639, 1.0000, 0.5237,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 0.7954,  ...

In [39]:
'''from torch._refs import nn


#定义神经网络结构类
class ConvDenoiser(nn.Module):
    def __init__(self):
        super(ConvDenoiser,self).__init__()
        #编码器
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride =1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride =1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, stride =1,padding=1)
        self.pool =nn.MaxPool( 2,2)
        #解码器
        self.t_conv1 = nn.ConvTranspose2d(8,8,kernel_size=3,stride=2,padding=1)
        self.t_conv2 = nn.ConvTranspose2d(8,16,kernel_size=3,stride=2,padding=1)
         self.t_conv3= nn.ConvTranspose2d(16,32,kernel_size=3,stride=2,padding=1)
         self.conv_out= nn.Conv2d(32,3,3,padding=1)

def forward(self, x):

#编码
        x = torch.relu(self.t_conv1(x))
        print('conv1 shape:',x.shape)
        x = self.pool(x)
        print('pool1 shape:',x.shape)

        x = torch.relu(self.t_conv2(x))
        print('conv2 shape:',x.shape)
        x = self.pool(x)
        print('pool2 shape:',x.shape)

        x = torch.relu(self.t_conv3(x))
        print('conv3 shape:',x.shape)
        x = self.pool(x)
        print('pool3 shape:',x.shape)

#解码
       x=torch.relu(self.conv1(x))
       print('conv1 shape:',x.shape)

        x=torch.relu(self.conv2(x))
       print('conv2 shape:',x.shape)

        x=torch.relu(self.conv3(x))
       print('conv3 shape:',x.shape)

       x=torch.sigmoid(self.conv_out(x))
       print('conv_out  shape:',x.shape)

       return  x


model = ConvDenoiser()
print(model)

x = torch.forward(torch.rand(1,3,68,68))
'''

import torch
import torch.nn as nn

class ConvDenoiser(nn.Module):
    def __init__(self):
        super(ConvDenoiser, self).__init__()
        # 编码层定义（示例，你可以根据需求调整通道数和核大小）
        self.t_conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.t_conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.t_conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # 池化层

        # 解码层定义（注意通道数要和编码对应）
        self.conv1 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=3, padding=1)
        self.conv_out = nn.Conv2d(3, 3, kernel_size=3, padding=1)  # 输出层

    def forward(self, x):
        # 编码
        x = torch.relu(self.t_conv1(x))
        print('conv1 shape:',x.shape)
        x = self.pool(x)
        print('pool1 shape:',x.shape)

        x = torch.relu(self.t_conv2(x))
        print('conv2 shape:',x.shape)
        x = self.pool(x)
        print('pool2 shape:',x.shape)

        x = torch.relu(self.t_conv3(x))
        print('conv3 shape:',x.shape)
        x = self.pool(x)
        print('pool3 shape:',x.shape)

        # 解码（缩进和编码部分保持一致）
        x = torch.relu(self.conv1(x))
        print('conv1 shape:',x.shape)

        x = torch.relu(self.conv2(x))
        print('conv2 shape:',x.shape)

        x = torch.relu(self.conv3(x))
        print('conv3 shape:',x.shape)

        x = torch.sigmoid(self.conv_out(x))
        print('conv_out  shape:',x.shape)

        return x

# 实例化模型并测试
model = ConvDenoiser()
print(model)

# 生成测试输入 [batch_size, channels, height, width]
test_input = torch.rand(1, 3, 68, 68)
# 正确调用 forward 方法
output = model(test_input)  # 等价于 model.forward(test_input)





ConvDenoiser(
  (t_conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (t_conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (t_conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_out): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
conv1 shape: torch.Size([1, 32, 68, 68])
pool1 shape: torch.Size([1, 32, 34, 34])
conv2 shape: torch.Size([1, 64, 34, 34])
pool2 shape: torch.Size([1, 64, 17, 17])
conv3 shape: torch.Size([1, 128, 17, 17])
pool3 shape: torch.Size([1, 128, 8, 8])
conv1 shape: torch.Size([1, 64, 8, 8])
conv2 shape: torch.Size([1, 32, 8, 8])
conv3 shape: torch.Size([1, 3

In [40]:
#指定损失函数
criterion = nn.MSELoss()
#指定优化器
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [44]:
#创建数据集
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import numpy as np
import torchvision.transforms as T

#检查是否有可用的GPU，有则使用GPU，否则用CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

#定义数据预处理
transforms=T.Compose([
    T.Resize((68,68)),
    T.ToTensor()
])

print("--------------正在创建的数据集------------")

#创建数据集


full_dataset=ImageDataset(image_dir ='../common/dataset/', transform=transform)

#计算训练集和验证集的大小
train_size = int(0.75 * len(full_dataset))
val_size = len(full_dataset) - train_size


#按照计算的数据大小，随机划分训练集和验证集
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

#设置训练书记的批次大小
batch_size=32

print('数据集创建完成，正在创建数据加载器...')


#创建训练集的数据加载器
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

#创建验证集的数据加载器
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False
)
for noisy_imgs, clean_imgs in train_loader:
    print(f'Noisy images batch shape: {noisy_imgs.shape}')
    print(f'Clean images batch shape: {clean_imgs.shape}')
    break

print('数据加载器创建完成，开始训练模型...')

--------------正在创建的数据集------------
数据集创建完成，正在创建数据加载器...
Noisy images batch shape: torch.Size([32, 3, 68, 68])
Clean images batch shape: torch.Size([32, 3, 68, 68])
数据加载器创建完成，开始训练模型...


In [52]:
#设置训练的epoch数量
num_epochs = 20
#设置噪声因子，勇于向图像添加噪声
noise_factor = 0.5
model.to(device)  #将模型发送到指定设备（CPU或GPU）
criterion = nn.MSELoss()
# 定义优化器（关键：之前缺少这一步）
optimizer = optim.Adam(model.parameters(), lr=1e-3)
#optimizer.to(device)  错误代码

for epoch in range(num_epochs):
    model.train()  #设置模型为训练模式
    running_loss = 0.0

    for noisy_imgs, clean_imgs in train_loader:
        noisy_imgs = noisy_imgs.to(device)  #将数据发送到指定设备
        clean_imgs = clean_imgs.to(device)

        #前向传播
        outputs = model(noisy_imgs)
        loss = criterion(outputs, clean_imgs)

        #反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * noisy_imgs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

conv1 shape: torch.Size([32, 32, 68, 68])
pool1 shape: torch.Size([32, 32, 34, 34])
conv2 shape: torch.Size([32, 64, 34, 34])
pool2 shape: torch.Size([32, 64, 17, 17])
conv3 shape: torch.Size([32, 128, 17, 17])
pool3 shape: torch.Size([32, 128, 8, 8])
conv1 shape: torch.Size([32, 64, 8, 8])
conv2 shape: torch.Size([32, 32, 8, 8])
conv3 shape: torch.Size([32, 3, 8, 8])
conv_out  shape: torch.Size([32, 3, 8, 8])


RuntimeError: The size of tensor a (8) must match the size of tensor b (68) at non-singleton dimension 3